Fonction à importer


In [377]:
# Core libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
# Sklearn functionality
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import export_text
from sklearn import preprocessing
# Keras functionality
import requests
# Ignore warnings
pd.options.mode.chained_assignment = None

Récupération des données

In [392]:
# Import data
stat_equipe_2023 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/Statistiques-Ligue-1#stats_squads_standard_for").content)[0]
matchs_ligue1_2023 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/calendrier/Scores-et-tableaux-Ligue-1#sched_2022-2023_13_1").content)[0]
matchs_ligue1_2022 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2021-2022/calendrier/Calendrier-et-resultats-2021-2022-Ligue-1#sched_all").content)[0]
matchs_ligue1_2021 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2020-2021/calendrier/Calendrier-et-resultats-2020-2021-Ligue-1#sched_all").content)[0]
matchs_ligue1_2020 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2019-2020/calendrier/Calendrier-et-resultats-2019-2020-Ligue-1#sched_2019-2020_13_1").content)[0]
matchs_ligue1_2019 = pd.read_html(requests.get("https://fbref.com/fr/comps/13/2018-2019/calendrier/Calendrier-et-resultats-2018-2019-Ligue-1#sched_all").content)[0]

Lier les données

In [393]:
list_concat = [matchs_ligue1_2019,matchs_ligue1_2020, matchs_ligue1_2021, matchs_ligue1_2022, matchs_ligue1_2023]
matchs_ligue1 = pd.concat(list_concat)

Créer les stats équipe pour chaque journée

In [401]:
stat_2019=[]
stat_2019.append(stat_equipe_2023[["Clt", "Équipe", "MJ", "V", "N", "D", "BM", "BE", "Pts","xG"]])
stat_2019[0][['MJ','V', 'N', 'D', 'BM', 'BE', 'Pts', 'xG','Pts/5','xG/5',"Domicile"]] = 0
stat_2019[0].head()
for k in range(37):
    stat_2019.append(stat_2019[k])
    stat_2019[k+1].loc[:, 'MJ'] += 1
    for i in range(len(stat_2019[k+1])):
        donnee_match = matchs_ligue1_2019.loc[(matchs_ligue1_2019["Sem."]==k+1) & ((matchs_ligue1_2019["Domicile"]==stat_2019[k].loc[i, 'Équipe'])| (matchs_ligue1_2019["Extérieur"]==stat_2019[k+1].loc[i, 'Équipe']))]
        if donnee_match["Domicile"]==stat_2019[k+1].loc[i, 'Équipe']:
            domicile = 1
        else:
            domicile = 0
        stat_2019[k].loc[i, 'Domicile'] = domicile
        if domicile:
            stat_2019[k+1].loc[i, 'V'] += donnee_match["Résultat"].iloc[0]
        
    

Nettoyer les données

In [381]:
# Enlever les colonnes inutiles
matchs_ligue1 = matchs_ligue1[matchs_ligue1["Tour"] != "Barrage 1/2 Relégation/Promotion France"]
matchs_trier= matchs_ligue1.drop(columns=["Tour","Sem.","Jour","Tribune","Arbitre","Notes","Rapport de match","Affluence","xG.1", "xG","Heure"],axis=1)
# Enlever les lignes inutiles
matchs_trier = matchs_trier[matchs_trier["Date"].notna()]
# Mettre les dates au bon format
matchs_trier["Date"] = pd.to_datetime(matchs_trier["Date"])
# Garder les matchs joués
matchs_joue = matchs_trier.dropna(subset=["Score"])
# Remettre les index à 0
matchs_joue.reset_index(drop=True, inplace=True)
# Ajouter les colonnes but_domicile et but_exterieur
matchs_joue["but_domicile"] = matchs_joue["Score"].str[0]
matchs_joue["but_exterieur"] = matchs_joue["Score"].str[2]
# garder les matchs à jouer
matchs_a_jouer = matchs_trier[matchs_trier["Score"].isna()]
# copier les colonnes Domicile et Extérieur dans les parties à jouer pour les retrouver après 
matchs_a_jouer["Domicile2"]=matchs_a_jouer["Domicile"]
matchs_a_jouer["Extérieur2"]=matchs_a_jouer["Extérieur"]
# Mettres les buts en type int
matchs_joue["but_domicile"] = matchs_joue["but_domicile"].astype(int)
matchs_joue["but_exterieur"] = matchs_joue["but_exterieur"].astype(int)
# Ajouter les colonnes nombre de but et écarts de but
matchs_joue["nombre_de_but"] = matchs_joue["but_domicile"] + matchs_joue["but_exterieur"]
matchs_joue["ecart_de_but"] = matchs_joue["but_domicile"] - matchs_joue["but_exterieur"]

# Ajouter les colonnes resultat_domicile et resultat_exterieur
    # matchs_joue["resultat_domicile"] = 0
    # matchs_joue["resultat_exterieur"] = 0
    # for i in range(len(matchs_joue)):
    #     if matchs_joue["but_domicile"][i] > matchs_joue["but_exterieur"][i]:
    #         matchs_joue["resultat_domicile"][i] = 2
    #         matchs_joue["resultat_exterieur"][i] = 0
    #     elif matchs_joue["but_domicile"][i] < matchs_joue["but_exterieur"][i]:
    #         matchs_joue["resultat_domicile"][i] = 0
    #         matchs_joue["resultat_exterieur"][i] = 2
    #     else:
    #         matchs_joue["resultat_domicile"][i] = 1
    #         matchs_joue["resultat_exterieur"][i] = 1
# Diviser la date en jour, mois et année
matchs_joue["jour"] = matchs_joue["Date"].dt.day
matchs_joue["mois"] = matchs_joue["Date"].dt.month
matchs_joue["annee"] = matchs_joue["Date"].dt.year
matchs_joue.drop(columns=["Date"],axis=1,inplace=True)
# Diviser la date en jour, mois et année pour les matchs à jouer
matchs_a_jouer["jour"] = matchs_a_jouer["Date"].dt.day
matchs_a_jouer["mois"] = matchs_a_jouer["Date"].dt.month
matchs_a_jouer["annee"] = matchs_a_jouer["Date"].dt.year
# Enlever les matchs covid
matchs_a_jouer = matchs_a_jouer.loc[matchs_a_jouer["annee"] == 2023]


Créer les données de test et d'entrainement

In [382]:
X = matchs_joue[["Domicile","Extérieur","jour","mois","annee"]]
y = matchs_joue[["nombre_de_but","ecart_de_but"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Encoder les colonnes Domicile et Extérieur pour les mettres en entier


In [383]:
cols = ['Domicile', 'Extérieur']
enc = preprocessing.LabelEncoder()
for col in cols:
    X_train[col] = X_train[col].astype('str')
    X_test[col] = X_test[col].astype('str')
    X_train[col] = enc.fit_transform(X_train[col])
    X_test[col] = enc.transform(X_test[col])
    matchs_a_jouer[col] = enc.fit_transform(matchs_a_jouer[col])

Création du modèle décision tree pour le nombre de but

In [384]:
# Build a logistic regression model
mode1_dt = DecisionTreeClassifier()
mode1_dt.fit(X_train, y_train["nombre_de_but"])
# Afficher l'arbre de décision
print("décision_tree pour écart de but \n",export_text(mode1_dt, feature_names=list(X_train)))
# Check the model performance with the training data
predictions1_dt_train = mode1_dt.predict(X_train)
print("prediction avec les training data:",accuracy_score(y_train["nombre_de_but"], predictions1_dt_train))
# Check the model performance with the test data
predictions1_dt_test = mode1_dt.predict(X_test)
print("prediction avec les test data:",accuracy_score(y_test["nombre_de_but"], predictions1_dt_test))
matchs_a_jouer["nombre_de_but"] = mode1_dt.predict(matchs_a_jouer[["Domicile","Extérieur","jour","mois","annee"]])

décision_tree pour écart de but 
 |--- mois <= 3.50
|   |--- Extérieur <= 2.50
|   |   |--- annee <= 2020.50
|   |   |   |--- mois <= 2.50
|   |   |   |   |--- mois <= 1.50
|   |   |   |   |   |--- Domicile <= 10.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- Domicile >  10.50
|   |   |   |   |   |   |--- jour <= 25.00
|   |   |   |   |   |   |   |--- class: 4
|   |   |   |   |   |   |--- jour >  25.00
|   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- mois >  1.50
|   |   |   |   |   |--- Domicile <= 24.00
|   |   |   |   |   |   |--- Extérieur <= 1.50
|   |   |   |   |   |   |   |--- jour <= 3.50
|   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |--- jour >  3.50
|   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |--- Extérieur >  1.50
|   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- Domicile >  24.00
|   |   |   |   |   |   |--- class: 3
|   |   |   |--- mois >  2.50
|   |   |   |   |--- jour

Création du model decision tree pour l'écart de but

In [385]:
# Build a decision tree model
mode1_dt = DecisionTreeClassifier()
mode1_dt.fit(X_train, y_train["ecart_de_but"])
# Afficher l'arbre de décision
print("décision_tree pour écart de but \n",export_text(mode1_dt, feature_names=list(X_train)))
# Check the model performance with the training data
predictions1_dt_train = mode1_dt.predict(X_train)
print("prediction avec les training data:",accuracy_score(y_train["ecart_de_but"], predictions1_dt_train))
# Check the model performance with the test data
predictions1_dt_test = mode1_dt.predict(X_test)
print("prediction avec les test data:",accuracy_score(y_test["ecart_de_but"], predictions1_dt_test))
matchs_a_jouer["ecart_de_but"] = mode1_dt.predict(matchs_a_jouer[["Domicile","Extérieur","jour","mois","annee"]])

décision_tree pour écart de but 
 |--- Extérieur <= 9.50
|   |--- Domicile <= 8.50
|   |   |--- Extérieur <= 0.50
|   |   |   |--- class: -1
|   |   |--- Extérieur >  0.50
|   |   |   |--- mois <= 6.50
|   |   |   |   |--- jour <= 4.00
|   |   |   |   |   |--- class: 2
|   |   |   |   |--- jour >  4.00
|   |   |   |   |   |--- Extérieur <= 4.50
|   |   |   |   |   |   |--- mois <= 3.50
|   |   |   |   |   |   |   |--- mois <= 2.50
|   |   |   |   |   |   |   |   |--- jour <= 28.00
|   |   |   |   |   |   |   |   |   |--- annee <= 2021.50
|   |   |   |   |   |   |   |   |   |   |--- annee <= 2020.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 2
|   |   |   |   |   |   |   |   |   |   |--- annee >  2020.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |   |   |   |   |--- annee >  2021.50
|   |   |   |   |   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |   |   |   |--- jour >  28.00
|   |   |   |   |   |   |   |   | 

Prévision des résultats

In [386]:
matchs_a_jouer["Score"] = ((matchs_a_jouer["nombre_de_but"]+matchs_a_jouer["ecart_de_but"])/2).astype(int).astype(str) + " - " + ((matchs_a_jouer["nombre_de_but"]-matchs_a_jouer["ecart_de_but"])/2).astype(int).astype(str)
matchs_prevu = matchs_a_jouer[["Date","Domicile2","Extérieur2","Score","nombre_de_but","ecart_de_but"]]
matchs_prevu.head(15)

,Date,Domicile2,Extérieur2,Score,nombre_de_but,ecart_de_but
287,2023-03-10,Lille,Lyon,2 - 2,4,0
288,2023-03-11,Auxerre,Rennes,2 - 3,5,-1
289,2023-03-11,Brest,Paris S-G,1 - 4,5,-3
290,2023-03-12,Clermont Foot,Lens,1 - 0,1,1
291,2023-03-12,Lorient,Troyes,1 - 1,2,0
292,2023-03-12,Ajaccio,Montpellier,0 - 0,0,0
293,2023-03-12,Nantes,Nice,0 - 0,0,0
294,2023-03-12,Angers,Toulouse,0 - 1,2,-1
295,2023-03-12,Monaco,Reims,1 - 3,5,-2
296,2023-03-12,Marseille,Strasbourg,2 - 0,2,2


Modele logisitic regression ecart de but

In [387]:
# Build a logistic regression model
mode1_lr = LogisticRegression(solver = 'liblinear', max_iter=1000, multi_class='ovr')
mode1_lr.fit(X_train, y_train["nombre_de_but"])
# Check the model performance with the training data
predictions1_lr_train = mode1_lr.predict(X_train)
print("prediction avec les training data:",accuracy_score(y_train["nombre_de_but"], predictions1_lr_train))
# Check the model performance with the test data
predictions1_lr_test = mode1_lr.predict(X_test)
print("prediction avec les test data:",accuracy_score(y_test["nombre_de_but"], predictions1_lr_test))
matchs_a_jouer["nombre_de_but"] = mode1_lr.predict(matchs_a_jouer[["Domicile","Extérieur","jour","mois","annee"]])


prediction avec les training data: 0.24829931972789115
prediction avec les test data: 0.21386138613861386


Modele logistic regression nombre de but

In [388]:
# Buill a logistic regression model for the ecart de but
mode2_lr = LogisticRegression(solver = 'liblinear', max_iter=1000, multi_class='ovr')
mode2_lr.fit(X_train, y_train["ecart_de_but"])
# Check the model performance with the training data
predictions2_lr_train = mode2_lr.predict(X_train)
print("prediction avec les training data:",accuracy_score(y_train["ecart_de_but"], predictions2_lr_train))
# Check the model performance with the test data
predictions2_lr_test = mode2_lr.predict(X_test)
print("prediction avec les test data:",accuracy_score(y_test["ecart_de_but"], predictions2_lr_test))
matchs_a_jouer["ecart_de_but"] = mode2_lr.predict(matchs_a_jouer[["Domicile","Extérieur","jour","mois","annee"]])


prediction avec les training data: 0.2755102040816326
prediction avec les test data: 0.2495049504950495


Prévision des résultat logistic regression


In [389]:
matchs_a_jouer["Score"] = ((matchs_a_jouer["nombre_de_but"]+matchs_a_jouer["ecart_de_but"])/2).astype(int).astype(str) + " - " + ((matchs_a_jouer["nombre_de_but"]-matchs_a_jouer["ecart_de_but"])/2).astype(int).astype(str)
matchs_prevu = matchs_a_jouer[["Date","Domicile2","Extérieur2","Score","nombre_de_but","ecart_de_but"]]
matchs_prevu.head(15)

,Date,Domicile2,Extérieur2,Score,nombre_de_but,ecart_de_but
287,2023-03-10,Lille,Lyon,1 - 1,3,0
288,2023-03-11,Auxerre,Rennes,1 - 1,3,0
289,2023-03-11,Brest,Paris S-G,1 - 1,3,0
290,2023-03-12,Clermont Foot,Lens,1 - 1,3,0
291,2023-03-12,Lorient,Troyes,1 - 1,3,0
292,2023-03-12,Ajaccio,Montpellier,1 - 1,3,0
293,2023-03-12,Nantes,Nice,1 - 1,3,0
294,2023-03-12,Angers,Toulouse,1 - 1,3,0
295,2023-03-12,Monaco,Reims,1 - 1,3,0
296,2023-03-12,Marseille,Strasbourg,1 - 1,3,0
